# IMPORTS

In [8]:
import os
from tqdm import tqdm
from azure.storage.blob import ContainerClient
from azure.core import exceptions
import yaml
from distutils.command.config import config
from tkinter import N
from azure.servicebus import ServiceBusClient, ServiceBusMessage, exceptions 
from copy import deepcopy
import pandas as pd
import time

import glob
import open3d as o3d
import numpy as np
import h5py
import re

# FUNCTIONS

In [9]:

def load_config(dir_config):
    with open(os.path.join(dir_config, "config.yaml"), "r") as yamlfile:
        return yaml.load(yamlfile, Loader=yaml.FullLoader)

In [10]:
def download_files(file_name, container_client, temp_dir="temp"):
    filepath=os.path.join(temp_dir,file_name)
    if not os.path.exists(filepath):
        if not container_client.exists(): 
                print("wrong connection_string")
        else:
            blobs=container_client.list_blobs()
            for blob in blobs:
                blob_name=blob.name
                if blob_name==file_name:
                    StorageStreamDownloader = container_client.download_blob(blob)
                    try:
                        file = open(filepath, 'wb')
                    except FileNotFoundError:
                        os.mkdir(temp_dir)
                        file = open(filepath, 'wb')
                    data=StorageStreamDownloader.readall()
                    print(f"saving locally blob: {blob_name}")
                    file.write(data)
                    file.close()
                    return filepath
    else:
        return filepath

In [11]:
def identification(pcds, variation, stl, stl_nums, container_client,service_bus_client):
    confidences=np.random.random((len(pcds),24))
    confidence=confidences[np.argmax(confidences[:,stl-1])]
    return confidence

In [12]:
def parse_labels(filepath):
    dict_labels={}
    text=""
    with open(filepath, "r") as file:
        text=file.read()
        for text_split in text.split("variant: "):
            if text_split:
                text_split_split=text_split.split("\n")
                #+re.match("labels: [",text_split_split[1])
                dict_labels[int(text_split_split[0])]=text_split_split[1].split("labels: ['")[1].split("']")[0].split("', '")
    return dict_labels


In [13]:
def get_parts(part_nums):
    file_names=[]
    for part_num in part_nums:
        prefix="part"
        part_name="%s%s.pcd"%(prefix,part_num)
        file_names.append(download_files(part_name, container_client,temp_dir="temp"))
    point_clouds=["pcd"]
    # print(file_names)
    return (file_names)

#### load pcd and random sample the points

In [93]:
def get_pcds_and_input(part_nums,model_size):
    file_names=get_parts(part_nums)
    pcds=[]
    for file in file_names:
        pcds.append(o3d.io.read_point_cloud(file))
    input=np.zeros((len(pcds),3,model_size))
    for count,pcd in enumerate(pcds):
        pcd.translate(-pcd.get_center())
        points_pcd=np.asarray(pcd.points)
        if len(points_pcd)<model_size: continue
        index_points_pcd=np.arange(len(points_pcd))
        np.random.shuffle(index_points_pcd)
        index_points_pcd=index_points_pcd[:model_size]
        points_sampled=points_pcd[index_points_pcd].T
        input[count,:,:]=points_sampled
    return pcds, input

### Load torch model

In [15]:
def load_model_variation(variation,var_load_option , variations, variations_NN):
    model_size=int(variations_NN["size"][int(variations[var_load_option,4])])
    print("# Variation: %s"%(variation+1))
    print(str(variations[var_load_option]))
    print("model_size: %s"%model_size)
    parser = argparse.ArgumentParser(description='Point Cloud Recognition')
    parser.add_argument('--k', type=int, default=20, metavar='N',
                            help='Num of nearest neighbors to use')
    parser.add_argument('--emb_dims', type=int, default=model_size, metavar='N',
                            help='Dimension of embeddings')
    parser.add_argument('--dropout', type=float, default=0.5,
                        help='initial dropout rate')
    args = parser.parse_args(args=[])
    trained_model=os.path.join(current_dir, "..", "TrainML", "trained", "variation-%s"%(variation+1),"models", "variation-%s_model.pth"%(variation+1))
    model=torch.nn.DataParallel(DGCNN_cls(args,24))
    load_model=torch.load(trained_model, map_location='cpu')
    model.load_state_dict(load_model)
    model.eval()
    return model, model_size

#### Permutate outputes to fit sorted labels

In [16]:
def permutation_matrix_outputs(var,dict_labels):
    #get standard sorted list
    permutation_matrix=np.zeros((len(dict_labels[1]),len(dict_labels[1])))
    numbers=[]
    for label in dict_labels[1]:
        numbers.append(int(label[-3:]))
    numbers.sort()

    # test=np.zeros((len(dict_labels[1])))
    for count_label, label in enumerate(dict_labels[var+1]):
        # test[count_label]=int(label[-3:])
        permutation_matrix[numbers.index(int(label[-3:])),count_label]=1
    return(permutation_matrix)


# EXECUTE

In [17]:
import os
import open3d as o3d

In [18]:
for file in os.listdir("temp"):
    print(file)
#     o3d.visualization.draw_geometries([o3d.io.read_point_cloud(os.path.join("temp",file))])


part0.pcd
part1.pcd
part10.pcd
part11.pcd
part12.pcd
part13.pcd
part14.pcd
part15.pcd
part16.pcd
part17.pcd
part18.pcd
part19.pcd
part2.pcd
part20.pcd
part21.pcd
part22.pcd
part23.pcd
part24.pcd
part25.pcd
part26.pcd
part27.pcd
part28.pcd
part29.pcd
part3.pcd
part30.pcd
part31.pcd
part32.pcd
part33.pcd
part34.pcd
part35.pcd
part36.pcd
part37.pcd
part38.pcd
part39.pcd
part4.pcd
part40.pcd
part41.pcd
part42.pcd
part43.pcd
part44.pcd
part45.pcd
part46.pcd
part47.pcd
part48.pcd
part5.pcd
part6.pcd
part7.pcd
part8.pcd
part9.pcd


In [106]:

pipename_local_to_data_prep = "lmdp"
pipename_data_prep_to_local= "dplm"
pipename_data_prep_to_ID_NN= "dpin"
pipename_data_prep_to_ID_reg= "dpir"
pipename_img_gen_to_train_reg= "imtr"
pipename_img_gen_to_train_ML= "imtm"
pipename_train_ML_to_train_ML= "tmtm"
pipename_local_to_img_gen= "lmim"
config=load_config("")
container_name=config["container_name"]+pipename_data_prep_to_ID_reg
container_client = ContainerClient.from_connection_string(config["azure_storage_connectionstring"], container_name)
save_path="temp"
service_bus_client=servicebus_client = ServiceBusClient.from_connection_string(conn_str=config['azure_messaging_connectionstring'])
container_name=config["container_name"]+pipename_data_prep_to_ID_reg
container_client = ContainerClient.from_connection_string(config["azure_storage_connectionstring"], container_name)


## Parse labels

In [107]:
dict_labels=parse_labels("label.txt")

In [108]:
organized_list=[]
numbers=[]
for label in dict_labels[1]:
    numbers.append(int(label[-3:]))
numbers.sort()
for num in numbers:
    organized_list.append("Bauteil-%s"%("{0:03}".format(num)))
print(organized_list)

['Bauteil-006', 'Bauteil-007', 'Bauteil-008', 'Bauteil-009', 'Bauteil-010', 'Bauteil-011', 'Bauteil-012', 'Bauteil-013', 'Bauteil-014', 'Bauteil-015', 'Bauteil-016', 'Bauteil-017', 'Bauteil-018', 'Bauteil-019', 'Bauteil-020', 'Bauteil-022', 'Bauteil-023', 'Bauteil-024', 'Bauteil-025', 'Bauteil-026', 'Bauteil-027', 'Bauteil-028', 'Bauteil-029', 'Bauteil-030']


In [109]:
dict_labels[38]

['Bauteil-028',
 'Bauteil-006',
 'Bauteil-008',
 'Bauteil-020',
 'Bauteil-013',
 'Bauteil-024',
 'Bauteil-018',
 'Bauteil-012',
 'Bauteil-029',
 'Bauteil-026',
 'Bauteil-016',
 'Bauteil-014',
 'Bauteil-027',
 'Bauteil-025',
 'Bauteil-022',
 'Bauteil-011',
 'Bauteil-009',
 'Bauteil-023',
 'Bauteil-007',
 'Bauteil-017',
 'Bauteil-015',
 'Bauteil-030',
 'Bauteil-010',
 'Bauteil-019']

## Load Dataset

In [110]:
from shutil import copy
current_dir= ""
model_src =os.path.join(current_dir, "..", "TrainML", "model.py")
model_path=os.path.join(current_dir,"model.py")


In [111]:
if not os.path.exists(model_path):
    copy(model_src,model_path)

### Load Model into torch

In [112]:
import torch
from model import DGCNN_cls
import argparse

### Load Variations

In [113]:

variations_NN={
        "radius":[1,4],
        "noise": [4,16],
        "view": [2,8],
        "batch": [16,64],
        "size": [1048,2048],
      }
      
length_variation=len(variations_NN)


#get variations
variations=np.zeros((2**length_variation,length_variation))
for num in range(2**length_variation):
    binary=np.asarray(list(str(bin(num))[2:]), dtype=int)
    row=binary
    if len(binary)<length_variation:
        row_add=np.zeros((1, length_variation-len(binary)), dtype=int).flatten()
        row=np.hstack((row_add,binary))
    variations[num]=row

In [114]:
index=np.nonzero(variations[:,4]!=0)
variation=index[0][-1]+1
print("# Variation: %s"%variation)
model_size=2048

# Variation: 32


### Test Model on on part

In [115]:
df =pd.read_csv("scans.csv",header=0)
stl_nums=np.sort(df["STL"].unique())

#### Load Model

#### input into model

In [116]:
max(dict_labels.keys())

40

In [117]:
time_result = np.zeros((max(dict_labels.keys()),len(df["Part"].unique())))
accuracy = np.zeros((max(dict_labels.keys())))#(len(dict_labels.keys())))
confusion_matrix=np.zeros((max(dict_labels.keys()),len(df["Part"].unique()),len(df["STL"].unique())))


In [96]:
# servicebus_client = ServiceBusClient.from_connection_string(conn_str=r"Endpoint=sb://messagingpipeline.servicebus.windows.net/;SharedAccessKeyName=RootManageSharedAccessKey;SharedAccessKey=yd4DxLLNyHoYZQGQ/RmYP3uZvDggm1RmBQuI7WUyGGY=")
container_name = "containerimtm"
container_client = ContainerClient.from_connection_string(config["azure_storage_connectionstring"], container_name)
save_path=os.path.join(".","save")
labels=[]
if not container_client.exists(): 
    print("container doesn't exist")
else:
    blobs=container_client.list_blobs()
    for blob in tqdm(blobs):
        blob_name=blob.name
        if blob_name.endswith(".h5"):
            print(blob_name)
        elif blob_name.endswith(".pcd"): 
            # print("deleting %s"%blob_name)
            container_client.delete_blob(blob, delete_snapshots="include")






0it [00:00, ?it/s]

ml_radius-0_noise-0_view-0_batch-0_size-0_variation-1_test_.h5






1it [00:02,  2.86s/it]

ml_radius-0_noise-0_view-0_batch-0_size-0_variation-1_train_.h5
ml_radius-0_noise-0_view-0_batch-0_size-1_variation-2_test_.h5
ml_radius-0_noise-0_view-0_batch-0_size-1_variation-2_train_.h5
ml_radius-0_noise-0_view-0_batch-1_size-0_variation-3_test_.h5
ml_radius-0_noise-0_view-0_batch-1_size-0_variation-3_train_.h5
ml_radius-0_noise-0_view-0_batch-1_size-1_variation-4_test_.h5
ml_radius-0_noise-0_view-0_batch-1_size-1_variation-4_train_.h5
ml_radius-0_noise-0_view-1_batch-0_size-0_variation-5_test_.h5
ml_radius-0_noise-0_view-1_batch-0_size-0_variation-5_train_.h5
ml_radius-0_noise-0_view-1_batch-0_size-1_variation-6_test_.h5
ml_radius-0_noise-0_view-1_batch-0_size-1_variation-6_train_.h5
ml_radius-0_noise-0_view-1_batch-1_size-0_variation-7_test_.h5
ml_radius-0_noise-0_view-1_batch-1_size-0_variation-7_train_.h5
ml_radius-0_noise-0_view-1_batch-1_size-1_variation-35_test_0.h5
ml_radius-0_noise-0_view-1_batch-1_size-1_variation-35_test_1.h5
ml_radius-0_noise-0_view-1_batch-1_size-1_va





64it [00:02,  2.00s/it]

...





65766it [31:35, 42.81it/s]



65771it [31:36, 42.47it/s]



65776it [31:36, 40.42it/s]



65781it [31:36, 40.93it/s]



65786it [31:36, 41.33it/s]



65791it [31:36, 38.57it/s]



65796it [31:36, 39.84it/s]



65801it [31:36, 40.52it/s]



65806it [31:36, 40.68it/s]



65811it [31:37, 39.96it/s]



65816it [31:37, 41.09it/s]



65821it [31:37, 40.27it/s]



65826it [31:37, 40.49it/s]



65831it [31:37, 35.34it/s]



65836it [31:37, 36.64it/s]



65841it [31:37, 38.19it/s]



65845it [31:37, 32.81it/s]



65849it [31:38, 34.58it/s]



65854it [31:38, 36.07it/s]



65859it [31:38, 37.15it/s]



65863it [31:38, 37.94it/s]



65868it [31:38, 39.27it/s]



65873it [31:38, 39.00it/s]



65877it [31:38, 39.09it/s]



65881it [31:38, 36.18it/s]



65885it [31:39, 30.56it/s]



65889it [31:39, 26.28it/s]



65892it [31:39, 24.87it/s]



65895it [31:39, 25.22it/s]



65898it [31:39, 25.20it/s]



65903it [31:39, 28.59it/s]



65907it [31:39, 31.12it/s]



65912it [31:40, 33.83it/s]



65916i

ml_radius-1_noise-1_view-1_batch-1_size-0_variation-31_test_.h5
ml_radius-1_noise-1_view-1_batch-1_size-0_variation-31_train_.h5






94311it [44:19, 44.11it/s]



94316it [44:19, 42.87it/s]



94321it [44:19, 41.20it/s]



94326it [44:19, 41.62it/s]



94331it [44:19, 41.24it/s]



94336it [44:20, 41.77it/s]



94341it [44:20, 40.93it/s]



94346it [44:20, 41.56it/s]



94351it [44:20, 37.23it/s]



94355it [44:20, 37.02it/s]



94359it [44:20, 33.63it/s]



94363it [44:20, 34.31it/s]



94368it [44:20, 35.85it/s]



94372it [44:21, 36.78it/s]



94377it [44:21, 38.13it/s]



94382it [44:21, 39.75it/s]



94387it [44:21, 39.54it/s]



94392it [44:21, 40.17it/s]



94397it [44:21, 39.93it/s]



94402it [44:21, 37.94it/s]



94407it [44:21, 38.98it/s]



94412it [44:22, 39.51it/s]



94417it [44:22, 39.41it/s]



94421it [44:22, 38.70it/s]



94425it [44:22, 35.00it/s]



94429it [44:22, 35.89it/s]



94433it [44:22, 36.29it/s]



94437it [44:22, 34.26it/s]



94441it [44:22, 35.78it/s]



94446it [44:22, 37.05it/s]



94450it [44:23, 37.65it/s]



94455it [44:23, 38.43it/s]



94459it [44:23, 35.25it/s]



94463i

ml_radius-1_noise-1_view-1_batch-1_size-1_variation-32_test_.h5
ml_radius-1_noise-1_view-1_batch-1_size-1_variation-32_train_.h5






122954it [56:50, 36.05it/s]

In [118]:

def rotation_matrix_3d(phi_x, phi_y, phi_z=0):
    rotation_x=np.asarray([
        [1.0,0,0],
        [0,np.cos(phi_x),-np.sin(phi_x)],
        [0,np.sin(phi_x),np.cos(phi_x)]
    ],dtype='float64')
    rotation_y=np.asarray([
        [np.cos(phi_y),0,np.sin(phi_y)],
        [0,1.0,0],
        [-np.sin(phi_y),0,np.cos(phi_y)]],dtype='float64')
    rotation_z=np.asarray([
        [np.cos(phi_z),-np.sin(phi_z),0],
        [np.sin(phi_z),np.cos(phi_z),0],
        [0,0,1.0]],dtype='float64')
    return np.matmul(np.matmul(rotation_x,rotation_y),rotation_z).astype('float64')


In [124]:
model, model_size = load_model_variation(39,5, variations,variations_NN)

# Variation: 40
[0. 0. 1. 0. 1.]
model_size: 2048


In [122]:
import h5py
f = h5py.File('ml_radius-0_noise-0_view-1_batch-1_size-1_variation-35_train_8.h5', 'r')
dset = f.get('data')
lset = f.get('label')
# print(np.asarray(dset)[1])

part_nums=[11]
_, inputs = get_pcds_and_input(part_nums, model_size)
print(inputs.shape)


idx=15

test=o3d.geometry.PointCloud()

test.points=o3d.utility.Vector3dVector(dset[idx*100])
test.rotate(rotation_matrix_3d(np.pi/2,np.pi/2,np.pi/2,))
test.translate(-test.get_center())
for input in inputs:
    pcd=o3d.geometry.PointCloud()
    print(input.T.shape)
    pcd.points=o3d.utility.Vector3dVector(input.T)
    pcd.translate(-pcd.get_center())
    # pcd.translate(test.get_center())
    torch_input_pcd=torch.tensor(np.asarray(pcd.points).T.reshape(1,3,2048)).float()

torch_input_test=torch.tensor(np.asarray(test.points).T.reshape(1,3,2048)).float()


output_test=model(torch_input_test).detach().numpy()
output_pcd=model(torch_input_pcd).detach().numpy()
print("output dataset model %s"%dict_labels[40][np.argmax(output_test)])
print("output datset %s"%dict_labels[35][lset[idx*100]])
print("output scan model %s"%dict_labels[40][np.argmax(output_pcd)])




for input in inputs:
    pcd.paint_uniform_color([0,1,0])
    test.paint_uniform_color([1,0,0])


    o3d.visualization.draw_geometries([test,pcd])

(1, 3, 2048)
(2048, 3)
output dataset model Bauteil-024
output datset Bauteil-024
output scan model Bauteil-023


In [90]:
pcd.get_center() #[-0.00994987, -0.01023108,  0.049349  ]

array([-0.00535698, -0.00850042,  0.06426845])

In [31]:
part_nums=[10,11]
pcds, inputs = get_pcds_and_input(part_nums, model_size)
print(inputs.shape)

for input in inputs:
    pcd=o3d.geometry.PointCloud()
    print(input.T.shape)
    pcd.points=o3d.utility.Vector3dVector(input.T)
    o3d.visualization.draw_geometries([pcd])

(2, 3, 2048)
(2048, 3)


(2048, 3)


1038it [00:43, 43.17it/s]

In [61]:
var=37
permutation_matrix=permutation_matrix_outputs(var,dict_labels)
model, model_size = load_model_variation(var,5, variations,variations_NN)

part_nums=[3,5]
_, inputs = get_pcds_and_input(part_nums, model_size)
print(inputs.shape)

for input in inputs:
    pcd=o3d.geometry.PointCloud()
    print(input.T.shape)
    pcd.points=o3d.utility.Vector3dVector(input.T)
    o3d.visualization.draw_geometries([pcd])

# pcds, input = get_pcds_and_input([9], model_size)
torch_input=torch.tensor(inputs).float()
# print(model(torch_input))
outputs=model(torch_input).detach().numpy()
# label_index=int(np.argmax(output_part))
# label_predicted=dict_labels[int(var+1)][label_index]

# o3d.visualization.draw_geometries(pcds)
# print(np.argmax(output))
for output in outputs:
    label_predicted=dict_labels[int(var+1)][np.argmax(output)]
    print(label_predicted)

# Variation: 38
[0. 0. 1. 0. 1.]
model_size: 2048
(2, 3, 2048)
(2048, 3)
(2048, 3)
Bauteil-006
Bauteil-006


In [128]:
for var in [40-1]:#range(len(variations)):
    permutation_matrix=permutation_matrix_outputs(var,dict_labels)
    model, model_size = load_model_variation(var,5, variations,variations_NN)
    score_eval=0
    count=0
    for count_STL, STL_num in enumerate(df["STL"].unique()):
        part_nums=df[df["STL"]==STL_num]["Part"].to_numpy()
        STL_name="Bauteil-%s"%("{0:03}".format(STL_num))
        print("par nums: %s"%part_nums)
        pcds, input = get_pcds_and_input(part_nums, model_size)
        torch_input=torch.tensor(input).float()
        t_id=time.time()
        output=model(torch_input).detach().numpy()
        id_time=(t_id-time.time())/len(part_nums)
        for count_output, output_part in enumerate(output):
            label_index=int(np.argmax(output_part))
            label_predicted=dict_labels[int(var+1)][label_index]

            # time_result[var,part_nums[count_output]]=id_time
            output_part=permutation_matrix@output_part[:len(df["STL"].unique())]
            confusion_matrix[1,part_nums[count_output],:]=output_part
            
            if STL_name==label_predicted: 
                score_eval+=1
            print("predicted: %s actual %s"%(label_predicted,STL_name))
            count+=1
    accuracy[var]=100*score_eval/count
    print("accuracy: %s percent"%(str(accuracy[var])))
    

# Variation: 40
[0. 0. 1. 0. 1.]
model_size: 2048
par nums: [0 1]
predicted: Bauteil-018 actual Bauteil-018
predicted: Bauteil-023 actual Bauteil-018
par nums: [2 4]
predicted: Bauteil-019 actual Bauteil-019
predicted: Bauteil-018 actual Bauteil-019
par nums: [3 5]
predicted: Bauteil-020 actual Bauteil-020
predicted: Bauteil-015 actual Bauteil-020
par nums: [6 7]
predicted: Bauteil-023 actual Bauteil-022
predicted: Bauteil-022 actual Bauteil-022
par nums: [8 9]
predicted: Bauteil-023 actual Bauteil-023
predicted: Bauteil-012 actual Bauteil-023
par nums: [10 11]
predicted: Bauteil-026 actual Bauteil-024
predicted: Bauteil-018 actual Bauteil-024
par nums: [12 13]
predicted: Bauteil-025 actual Bauteil-025
predicted: Bauteil-015 actual Bauteil-025
par nums: [14 15]
predicted: Bauteil-015 actual Bauteil-026
predicted: Bauteil-015 actual Bauteil-026
par nums: [16 17]
predicted: Bauteil-015 actual Bauteil-027
predicted: Bauteil-015 actual Bauteil-027
par nums: [18 19]
predicted: Bauteil-011 a

In [ ]:
time_df=pd.DataFrame(data=time_result, index=np.arange(start=1,stop=33))
accuracy_df=pd.DataFrame(data=accuracy, index=np.arange(start=1,stop=33), columns=["Accuracy"])
time_df["average over all parts"] = time_df. mean(axis=1)
results_path=os.path.join(current_dir,"results")
time_df.to_csv(os.path.join(results_path, "time_results.csv"))
accuracy_df.to_csv(os.path.join(results_path, "accuracy_results.csv"))

In [ ]:
accuracy_df

NameError: name 'accuracy_df' is not defined

## Test Model on Dataset

In [ ]:
mode="ml"
accuracy=np.zeros((16,len(stl_nums),len(stl_nums)))
if mode=="ml":
    accuracy=np.zeros((1,len(stl_nums),len(stl_nums)))

print(accuracy.shape)


(1, 24, 24)


Main loop 

download all parts

In [ ]:
# variation=1
# for stl_i, stl_num in enumerate(stl_nums[:1]):
    
#     for file_name in file_names:
#         print("download here")
#         # point_clouds.append(o3d.io.read_point_cloud(file_name))
#     print(point_clouds)
#     confidence=identification(point_clouds,variation, stl_i,stl_nums, container_client, service_bus_client)
#     print(confidence)
#     accuracy[variation-1,stl_i-1,:]=confidence

download here
download here
['pcd']
[3.81467170e-01 1.35964931e-01 6.48247278e-04 3.05368927e-01
 1.84905470e-02 2.52611643e-01 4.28609938e-01 3.30795339e-01
 7.98416018e-02 4.48178644e-01 2.95342793e-01 4.23999533e-01
 9.94964508e-01 6.57827402e-01 5.16157760e-01 7.79339476e-01
 8.77269000e-01 6.39586737e-01 1.00775900e-01 3.11890503e-01
 7.05590193e-01 5.48577045e-01 6.83105939e-01 2.91405448e-01]


#load excel